# Pré processamento da coluna SUMMARY

In [49]:
import pandas as pd
import unicodedata
import os


In [ ]:
try:
    df = pd.read_csv(
        '../data/new_tasks.csv',
        encoding='utf-8',
        sep=",",
    )
    print(f"CSV carregado com engine C! {len(df)} linhas encontradas.")

except Exception as e:
    print(f"Erro ao ler CSV: {e}")

# Converter colunas de data para datetime
df['DTSTART'] = pd.to_datetime(df['DTSTART'])
df['DTEND'] = pd.to_datetime(df['DTEND'])
df['CREATED'] = pd.to_datetime(df['CREATED'])
print("Colunas de data convertidas para datetime")

CSV carregado com engine C! 4463 linhas encontradas.
✅ Colunas de data convertidas para datetime


In [51]:
# Verificar informações básicas do dataset
print("=== INFORMAÇÕES BÁSICAS DO DATASET ===")
print(f"Shape: {df.shape}")
print(f"Colunas: {df.columns.tolist()}")
print(f"Tipos de dados:\n{df.dtypes}")

# Informações apenas da coluna SUMMARY
print(f"\nResumo da coluna SUMMARY:")
print(f"  - Tipo de dado: {df['SUMMARY'].dtype}")
print(f"  - Valores únicos: {df['SUMMARY'].nunique()}")
print(f"  - Valores nulos: {df['SUMMARY'].isnull().sum()}")
print(f"  - Primeiros 5 valores:\n{df['SUMMARY'].head()}")

=== INFORMAÇÕES BÁSICAS DO DATASET ===
Shape: (4463, 6)
Colunas: ['SUMMARY', 'DTSTART', 'DTEND', 'CALENDAR', 'DURATION', 'CREATED']
Tipos de dados:
SUMMARY             object
DTSTART     datetime64[ns]
DTEND       datetime64[ns]
CALENDAR            object
DURATION           float64
CREATED     datetime64[ns]
dtype: object

Resumo da coluna SUMMARY:
  - Tipo de dado: object
  - Valores únicos: 372
  - Valores nulos: 0
  - Primeiros 5 valores:
0     🍳
1     🧋
2    🍽️
3     🥪
4     🍌
Name: SUMMARY, dtype: object


### Análise

In [52]:
print("=== ANÁlISE DA COLUNA SUMMARY ===")

print(f"Quantidade de valores únicos: {df['SUMMARY'].nunique()}")


=== ANÁlISE DA COLUNA SUMMARY ===
Quantidade de valores únicos: 372


### Corrigindo os valores que estão com EMOJI

In [53]:
print(df['SUMMARY'].head(8))

# 🧋: lanche da manha
# 🍽️ e antes das 18:00: almoco
# 🥪: lanche da tarde
# 🍌: pre-treino
# 🍽️ e exatamente ou depois das 18:00: jantar

df.loc[df['SUMMARY'] == '🍳', 'SUMMARY'] = 'cafe da manha'
df.loc[df['SUMMARY'] == '🧋', 'SUMMARY'] = 'lanche da manha'
# Almoço: emoji 🍽️ e começa antes das 18h
df.loc[(df['SUMMARY'] == '🍽️') & (pd.to_datetime(df['DTSTART']).dt.hour < 18), 'SUMMARY'] = 'almoco'
df.loc[df['SUMMARY'] == '🥪', 'SUMMARY'] = 'lanche da tarde'
df.loc[df['SUMMARY'] == '🍌', 'SUMMARY'] = 'pre-treino'
df.loc[(df['SUMMARY'] == '🍽️') & (pd.to_datetime(df['DTSTART']).dt.hour >= 18), 'SUMMARY'] = 'jantar'

print(df['SUMMARY'].head(8))


0     🍳
1     🧋
2    🍽️
3     🥪
4     🍌
5    🍽️
6     🍳
7     🧋
Name: SUMMARY, dtype: object
0      cafe da manha
1    lanche da manha
2             almoco
3    lanche da tarde
4         pre-treino
5             jantar
6      cafe da manha
7    lanche da manha
Name: SUMMARY, dtype: object


## Limpando

In [54]:
def normalize_text(text):
    if pd.isnull(text):
        return ""
    # Lowercase
    text = text.lower()
    # Remove accents
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    # Remove extra spaces
    text = ' '.join(text.split())
    return text

# Avaliação antes da normalização
print("=== Avaliação da coluna SUMMARY antes da normalização ===")
print(f"Valores únicos: {df['SUMMARY'].nunique()}")
print(f"Exemplos aleatórios: {df['SUMMARY'].drop_duplicates().sample(5, random_state=42).tolist()}")

df['SUMMARY'] = df['SUMMARY'].apply(normalize_text)

# Avaliação depois da normalização
print("\n=== Avaliação da coluna SUMMARY depois da normalização ===")
print(f"Valores únicos: {df['SUMMARY'].nunique()}")
print(f"Exemplos aleatórios: {df['SUMMARY'].drop_duplicates().sample(5, random_state=42).tolist()}")


=== Avaliação da coluna SUMMARY antes da normalização ===
Valores únicos: 373
Exemplos aleatórios: ['Pulo Big Jump', 'INF1301 - Relatório Plano de Ação', 'Zion', 'Niver Giovanna Lopes', 'Avaliação 4 Álgebra']

=== Avaliação da coluna SUMMARY depois da normalização ===
Valores únicos: 369
Exemplos aleatórios: ['festa niver', 'p1 inf1018', 'zion', 'formatura renata', 'tete 2 maple algebra']


### Dropando valores que não são necessários

In [55]:
print('Numero de linhas antes de retirar RescueTime:', len(df))
df = df[df['CALENDAR'] != 'RescueTime']
print('Numero de linhas depois de retirar RescueTime:', len(df))

Numero de linhas antes de retirar RescueTime: 4463
Numero de linhas depois de retirar RescueTime: 4418


In [56]:
print('Número de linhas antes de retirar o Rotina: ', len(df))
df = df[df['CALENDAR'] != 'Rotina']
print('Número de linhas depois de retirar o Rotina: ', len(df))

Número de linhas antes de retirar o Rotina:  4418
Número de linhas depois de retirar o Rotina:  4372


In [57]:
print('Número de linhas antes de retirar parabens!: ', len(df))
df = df[df['SUMMARY'] != 'parabens']
print('Número de linhas depois de retirar parabens!: ', len(df))

Número de linhas antes de retirar parabens!:  4372
Número de linhas depois de retirar parabens!:  4372


### Tratando valores da coluna SUMMARY que possuem dois valores na coluna CALENDAR

In [58]:
# Coluna SUMMARY possui 30 valores que tem duas variações em 'CALENDAR'
print("=== TRATANDO Duplicatas da coluna SUMMARY ===")
df['CALENDAR'] = df['CALENDAR'].replace('PUC 👻', 'PUC')

# Corrigindo os valores duplicados de 'Academia'
df.loc[(df['SUMMARY'] == 'academia') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Exercícios' # 2
print(f'Valores de CALENDAR para "Academia": {df.loc[df["SUMMARY"] == "academia", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de 'Aldeia'
df.loc[(df['SUMMARY'] == 'aldeia') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Social' # 2
print(f'Valores de CALENDAR para "Aldeia": {df.loc[df["SUMMARY"] == "aldeia", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de 'Alessandra'
df.loc[(df['SUMMARY'] == 'alessandra') & (df['CALENDAR'] == 'Compromisso'), 'CALENDAR'] = 'Saúde' # 2
print(f'Valores de CALENDAR para "Alessandra": {df.loc[df["SUMMARY"] == "alessandra", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de 'Aula'
df.loc[(df['SUMMARY'] == 'aula') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'PUC' # 2
print(f'Valores de CALENDAR para "Aula": {df.loc[df["SUMMARY"] == "aula", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de 'BD: T2 G2'
df.loc[(df['SUMMARY'] == 'bd: t2 g2') & (df['CALENDAR'] == 'PUC'), 'CALENDAR'] = 'Exames' # 2
print(f'Valores de CALENDAR para "BD: T2 G2": {df.loc[df["SUMMARY"] == "bd: t2 g2", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de 'BIRUSAMBA DE VERÃO'
df.loc[(df['SUMMARY'] == 'birusamba de verao'), 'CALENDAR'] = 'Social' # 2
print(f'Valores de CALENDAR para "BIRUSAMBA DE VERÃO": {df.loc[df["SUMMARY"] == "birusamba de verao", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de De-Para
df.loc[(df['SUMMARY'] == 'de-para') & (df['CALENDAR'] == 'Exames'), 'CALENDAR'] = 'PUC' # 2
print(f'Valores de CALENDAR para "De-Para": {df.loc[df["SUMMARY"] == "de-para", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de 'Muay Thai'
df.loc[(df['SUMMARY'] == 'muay thai') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Exercícios' # 2
print(f'Valores de CALENDAR para "Muay Thai": {df.loc[df["SUMMARY"] == "muay thai", "CALENDAR"].unique()}')

# Corrigindo valores duplicados de 'Psicóloga'
df.loc[(df['SUMMARY'] == 'psicologa') & (df['CALENDAR'] == 'Compromisso'), 'CALENDAR'] = 'Saúde' # 2
print(f'Valores de CALENDAR para "Psicóloga": {df.loc[df["SUMMARY"] == "psicologa", "CALENDAR"].unique()}')

# Corrigindo valores duplicados de 'SO: P1' # 2
df.loc[(df['SUMMARY'] == 'so: p1') & (df['CALENDAR'] == 'PUC'), 'CALENDAR'] = 'Exames' # 2
print(f'Valores de CALENDAR para "SO: P1": {df.loc[df["SUMMARY"] == "so: p1", "CALENDAR"].unique()}')

# Corrigindo valores duplicados de 'Itaipava'
df.loc[(df['SUMMARY'] == 'itaipava') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Social' # 2
print(f'Valores de CALENDAR para "Itaipava": {df.loc[df["SUMMARY"] == "itaipava", "CALENDAR"].unique()}')

=== TRATANDO Duplicatas da coluna SUMMARY ===
Valores de CALENDAR para "Academia": ['Exercícios']
Valores de CALENDAR para "Aldeia": ['Social']
Valores de CALENDAR para "Alessandra": ['Saúde']
Valores de CALENDAR para "Aula": ['PUC']
Valores de CALENDAR para "BD: T2 G2": ['Exames']
Valores de CALENDAR para "BIRUSAMBA DE VERÃO": ['Social']
Valores de CALENDAR para "De-Para": ['PUC']
Valores de CALENDAR para "Muay Thai": ['Exercícios']
Valores de CALENDAR para "Psicóloga": ['Saúde']
Valores de CALENDAR para "SO: P1": ['Exames']
Valores de CALENDAR para "Itaipava": ['Social']


In [59]:
print("=== ENCONTRO DE DUPLICATAS DA COLUNA SUMMARY ===")
duplicates = df[df.duplicated(subset='SUMMARY', keep=False)]
# Seleciona duplicatas de SUMMARY que aparecem em mais de um CALENDAR
duplicates_sum_cal = (
    duplicates[['SUMMARY', 'CALENDAR']]
    .drop_duplicates()
    .groupby('SUMMARY')
    .filter(lambda x: x['CALENDAR'].nunique() > 1)
    .sort_values(by=['SUMMARY', 'CALENDAR'], ascending=[True, True])
)
print(f"Quantidade de duplicatas encontradas: {duplicates_sum_cal.shape[0]}")
print(f"Duplicatas encontradas:\n{duplicates_sum_cal}")

=== ENCONTRO DE DUPLICATAS DA COLUNA SUMMARY ===
Quantidade de duplicatas encontradas: 2
Duplicatas encontradas:
     SUMMARY     CALENDAR
2     almoco  Alimentação
2770  almoco      Pessoal


## Verificando valores parecidos

### Passando o valor 'Alessandra' da coluna SUMMARY para 'Psicóloga'

In [60]:
print(f"Quantidade de valores únicos antes da unificação: {df['SUMMARY'].nunique()}")

print('=== TROCANDO DE "alessandra" para "psiquiatra" ===')
df.loc[df['SUMMARY'] == 'alessandra', 'SUMMARY'] = 'psiquiatra'

print(f"Quantidade de valores únicos depois da unificação: {df['SUMMARY'].nunique()}")


psi_values = df[df['SUMMARY'].str.lower().str.startswith('psi', na=False)]['SUMMARY'].unique()
print(f"\nValores que começam com 'psi' antes da unificação: {psi_values}")

print('=== TROCANDO DE "psicologa" para "psicologo" ===')
df.loc[df['SUMMARY'] == 'psicologa', 'SUMMARY'] = 'psicologo'

psi_values = df[df['SUMMARY'].str.lower().str.startswith('psi', na=False)]['SUMMARY'].unique()
print(f"Valores que começam com 'psi' depois da unificação: {psi_values}")


Quantidade de valores únicos antes da unificação: 324
=== TROCANDO DE "alessandra" para "psiquiatra" ===
Quantidade de valores únicos depois da unificação: 323

Valores que começam com 'psi' antes da unificação: ['psiquiatra' 'psicologa' 'psicologo']
=== TROCANDO DE "psicologa" para "psicologo" ===
Valores que começam com 'psi' depois da unificação: ['psiquiatra' 'psicologo']


## Melhorando valores com INF, MAT ou ADM no inicio

### Verificando todos os valores que possuem inf, mat ou adm no inicio

In [61]:
print('=== Avaliando valores que começam com "inf", "mat" ou "adm"')

inf_mat_values = df.loc[
    df['SUMMARY'].str.lower().str.startswith('adm', na=False) |
    df['SUMMARY'].str.lower().str.startswith('inf', na=False) |
    df['SUMMARY'].str.lower().str.startswith('mat', na=False),
    'SUMMARY'
]
print(f'Quantidade de valores únicos: {inf_mat_values.nunique()}')

print(f'Valores:\n{inf_mat_values.value_counts()}')

=== Avaliando valores que começam com "inf", "mat" ou "adm"
Quantidade de valores únicos: 33
Valores:
SUMMARY
inf1410 - gerenc proj inf                     39
adm1019 - intr. financas                      38
inf1629 - p. eng. de software                 38
mat4162 - calculo ii                          38
inf1608 - analise numerica                    38
inf1307 - games                               36
inf1316 - so                                  36
inf1771 - ia                                  36
inf1403 - ihc                                 35
inf1022 - als                                 35
inf1350 - prog sis reat                       34
inf1027 - teste e medicao de software         33
inf1036 - prob comp                           32
inf1640 - redes de computadores               31
inf1636 - programacao orientada a objetos     31
inf1631 - estruturas discretas                31
inf1028 - projeto e construcao de sistemas    30
inf1721 - aa                                  30
inf1032 

### Retirando INF2471 - Prob Comp

In [62]:
print('Número de linhas antes de retirar inf2471 - prob comp: ', len(df))
df = df[df['SUMMARY'] != 'inf2471 - prob comp']
print('Número de linhas depois de retirar inf2471 - prob comp: ', len(df))


Número de linhas antes de retirar inf2471 - prob comp:  4372
Número de linhas depois de retirar inf2471 - prob comp:  4370


### Retirando MAT1162 - CALCULO A VARIAS VARIAVEIS I

In [63]:
print('Número de linhas antes de retirar mat1162 - calculo a varias variaveis i: ', len(df))
df = df[df['SUMMARY'] != 'mat1162 - calculo a varias variaveis i']
print('Número de linhas depois de retirar mat1162 - calculo a varias variaveis i: ', len(df))

Número de linhas antes de retirar mat1162 - calculo a varias variaveis i:  4370
Número de linhas depois de retirar mat1162 - calculo a varias variaveis i:  4367


In [ ]:
# Corrigir primeiro código que já estava sendo tratado
df.loc[df['SUMMARY'] == 'inf1410 - gerenc proj inf', 'SUMMARY'] = 'inf1410 - gerencia de projetos de informatica'

# INF1629 - P. Eng. de Software
df.loc[df['SUMMARY'] == 'inf1629 - p. eng. de software', 'SUMMARY'] = 'inf1629 - principios de engenharia de software'

# INF1314 - Startup IA
df.loc[df['SUMMARY'] == 'inf1314 - startup ia', 'SUMMARY'] = 'inf1314 - gerando startup com inteligencia artificial'

# INF1771 - IA
df.loc[df['SUMMARY'] == 'inf1771 - ia', 'SUMMARY'] = 'inf1771 - inteligencia artificial'

# INF1307 - Games
df.loc[df['SUMMARY'] == 'inf1307 - games', 'SUMMARY'] = 'inf1307 - desenvolvimento de jogos'

# INF1316 - SO
df.loc[df['SUMMARY'] == 'inf1316 - so', 'SUMMARY'] = 'inf1316 - sistemas operacionais'

# INF1403 - IHC
df.loc[df['SUMMARY'] == 'inf1403 - ihc', 'SUMMARY'] = 'inf1403 - interacao humano computador'

# INF1022 - ALS
df.loc[df['SUMMARY'] == 'inf1022 - als', 'SUMMARY'] = 'inf1022 - analisadores lexicos e sintaticos'

# INF1350 - Prog Sis Reat
df.loc[df['SUMMARY'] == 'inf1350 - prog sis reat', 'SUMMARY'] = 'inf1350 - programacao de sistemas reativos'

# INF1036 - Prob Comp
df.loc[df['SUMMARY'] == 'inf1036 - prob comp', 'SUMMARY'] = 'inf1036 - probabilidade computacional'

# INF1721 - AA
df.loc[df['SUMMARY'] == 'inf1721 - aa', 'SUMMARY'] = 'inf1721 - analise de algoritmos'

# INF1010 - EDA
df.loc[df['SUMMARY'] == 'inf1010 - eda', 'SUMMARY'] = 'inf1010 - estruturas de dados avançadas'

# INF1338 - LLM
df.loc[df['SUMMARY'] == 'inf1338 - llm', 'SUMMARY'] = 'inf1338 - large language models'

# INF1022 - Analisadores Léx. e Sint.
df.loc[df['SUMMARY'] == 'inf1022 - analisadores lex. e sint.', 'SUMMARY'] = 'inf1022 - analisadores lexicos e sintaticos'

df.loc[df['SUMMARY'] == 'adm1019 - intr. financas', 'SUMMARY'] = 'adm1019 - introducao a finanças'

print('Expansão de abreviações concluída - tudo em minúsculo!')

✅ Expansão de abreviações concluída - tudo em minúsculo!


### Verificando novamente os valores com INF, MAT ou ADM no inicio

In [65]:
print('=== Avaliando valores que começam com "inf", "mat" ou "adm"')

inf_mat_values = df.loc[
    df['SUMMARY'].str.lower().str.startswith('adm', na=False) |
    df['SUMMARY'].str.lower().str.startswith('inf', na=False) |
    df['SUMMARY'].str.lower().str.startswith('mat', na=False),
    'SUMMARY'
]
print(f'Quantidade de valores únicos: {inf_mat_values.nunique()}')

print(f'Valores:\n{inf_mat_values.value_counts()}')

=== Avaliando valores que começam com "inf", "mat" ou "adm"
Quantidade de valores únicos: 30
Valores:
SUMMARY
inf1410 - gerencia de projetos de informatica            39
inf1022 - analisadores lexicos e sintaticos              39
adm1019 - introducao a finanças                          38
inf1629 - principios de engenharia de software           38
mat4162 - calculo ii                                     38
inf1608 - analise numerica                               38
inf1771 - inteligencia artificial                        36
inf1316 - sistemas operacionais                          36
inf1307 - desenvolvimento de jogos                       36
inf1403 - interacao humano computador                    35
inf1350 - programacao de sistemas reativos               34
inf1027 - teste e medicao de software                    33
inf1036 - probabilidade computacional                    32
inf1631 - estruturas discretas                           31
inf1636 - programacao orientada a objetos         

## Verbalizar tarefas e anonimizar pessoas (com llm)

In [66]:
# Exportar tuplas únicas de (SUMMARY, CALENDAR) para CSV
# df[['SUMMARY', 'CALENDAR']].drop_duplicates().to_csv('../data/summary_calendar_unique.csv', index=False, encoding='utf-8')

### Colocando as tarefas nos (SUMMARY, CALENDAR) únicos

In [ ]:
# Carregar as tasks verbalizadas e integrar com o dataset original
print("=== INTEGRANDO TASKS VERBALIZADAS EM INGLÊS ===")

# Carregar o arquivo com as tasks verbalizadas
formatted_tasks_df = pd.read_csv('../data/fixed/formatted_tasks.csv', encoding='utf-8')
print(f"Tasks verbalizadas carregadas: {len(formatted_tasks_df)} registros")

# Criar um dicionário para mapear (SUMMARY, CALENDAR) -> TASK_EN
task_mapping = {}
for _, row in formatted_tasks_df.iterrows():
    key = (row['SUMMARY'], row['CALENDAR'])
    task_mapping[key] = row['TASK_EN']

print(f"Mapeamento criado para {len(task_mapping)} tuplas únicas")

# Adicionar a coluna TITLE ao dataset original (usando TASK_EN)
df['TITLE'] = df.apply(lambda row: task_mapping.get((row['SUMMARY'], row['CALENDAR']), ''), axis=1)

# Verificar quantos registros foram mapeados
mapped_count = len(df[df['TITLE'] != ''])
unmapped_count = len(df[df['TITLE'] == ''])

print(f"\nResultados do mapeamento:")
print(f"Registros com título atribuído: {mapped_count}")
print(f"Registros sem título: {unmapped_count}")
print(f"Taxa de sucesso: {(mapped_count / len(df) * 100):.2f}%")

# Mostrar exemplos dos dados integrados
print(f"\n=== EXEMPLOS DOS DADOS INTEGRADOS ===")
print(df[['SUMMARY', 'CALENDAR', 'TITLE']].head(10))

# Verificar se há registros não mapeados
if unmapped_count > 0:
    print(f"\n=== REGISTROS NÃO MAPEADOS ===")
    # Mostrar todos os não mapeados
    unmapped_df = df[df['TITLE'] == ''][['SUMMARY', 'CALENDAR']].drop_duplicates()
    print(f"Tuplas únicas não mapeadas: {len(unmapped_df)}")
    print(unmapped_df)

    # Remover registros sem TITLE usando a mesma lógica do unmapped_count
    print(f"\n=== REMOVENDO REGISTROS SEM TÍTULO ===")
    print(f"Registros antes da remoção: {len(df)}")
    df = df[df['TITLE'] != '']  # Filtra apenas registros COM título
    print(f"Registros após remoção: {len(df)}")
    print(f"Removidos {unmapped_count} registros sem título")
else:
    print("\nTodos os registros já possuem título!")

=== INTEGRANDO TASKS VERBALIZADAS EM INGLÊS ===
Tasks verbalizadas carregadas: 320 registros
Mapeamento criado para 320 tuplas únicas

Resultados do mapeamento:
✅ Registros com título atribuído: 4339
❌ Registros sem título: 28
📊 Taxa de sucesso: 99.36%

=== EXEMPLOS DOS DADOS INTEGRADOS ===
           SUMMARY     CALENDAR                    TITLE
0    cafe da manha  Alimentação           Have breakfast
1  lanche da manha  Alimentação     Have a morning snack
2           almoco  Alimentação               Have lunch
3  lanche da tarde  Alimentação  Have an afternoon snack
4       pre-treino  Alimentação         Take pre-workout
5           jantar  Alimentação              Have dinner
6    cafe da manha  Alimentação           Have breakfast
7  lanche da manha  Alimentação     Have a morning snack
8           almoco  Alimentação               Have lunch
9  lanche da tarde  Alimentação  Have an afternoon snack

=== REGISTROS NÃO MAPEADOS ===
Tuplas únicas não mapeadas: 27
                  

In [ ]:
# Verificar se todos os registros agora possuem TITLE
print("=== VERIFICAÇÃO FINAL ===")
print(f"Total de registros no dataset: {len(df)}")
print(f"Registros com TITLE vazio: {len(df[df['TITLE'] == ''])}")
print(f"Registros com TITLE preenchido: {len(df[df['TITLE'] != ''])}")

if len(df[df['TITLE'] == '']) == 0:
    print("SUCESSO: Todos os registros possuem TITLE!")
else:
    print("ERRO: Ainda há registros sem TITLE")

=== VERIFICAÇÃO FINAL ===
Total de registros no dataset: 4339
Registros com TITLE vazio: 0
Registros com TITLE preenchido: 4339
✅ SUCESSO: Todos os registros possuem TITLE!


## Verificar a coluna TITLE

In [69]:
# A coluna TITLE já vem formatada em inglês do arquivo formatted_tasks.csv
# Não é necessário aplicar normalização adicional
print("=== VERIFICAÇÃO DA COLUNA TITLE ===")
print(f"Valores únicos na coluna TITLE: {df['TITLE'].nunique()}")
print(f"Exemplos de títulos:")
print(df['TITLE'].drop_duplicates().sample(5, random_state=42).tolist())

=== VERIFICAÇÃO DA COLUNA TITLE ===
Valores únicos na coluna TITLE: 270
Exemplos de títulos:
['Take Modular Programming exam 1', 'Take Calculus exam 3', 'Take database test 3', 'Do bioimpedance', 'Attend inf1383 - Databases class']


## Descartar tarefas que cobrem o dia inteiro

In [70]:
print('=== DESCARTANDO TAREFAS QUE COBREM O DIA INTEIRO ===')
    # Filtrar tarefas que começam e terminam exatamente às 00:00 e duram 24 horas
mask = (
    (pd.to_datetime(df['DTSTART']).dt.hour == 0) &
    (pd.to_datetime(df['DTSTART']).dt.minute == 0) &
    (pd.to_datetime(df['DTEND']).dt.hour == 0) &
    (pd.to_datetime(df['DTEND']).dt.minute == 0) &
    (df['DURATION'] == 24.0)
)
print(f'Tarefas que cobrem o dia inteiro encontradas: {mask.sum()}')
df = df[~mask]
print(f'Total de registros após descartar tarefas de 24h: {len(df)}')


=== DESCARTANDO TAREFAS QUE COBREM O DIA INTEIRO ===
Tarefas que cobrem o dia inteiro encontradas: 304
Total de registros após descartar tarefas de 24h: 4035


# Exportar DataFrame completo para CSV

In [ ]:
# # Salvar dataset processado
print("=== SALVANDO DATASET PROCESSADO ===")

# Salvar como CSV processado
output_path = '../data/tasks_summary.csv'
df.to_csv(output_path, index=False, encoding='utf-8')
print(f"Dataset processado salvo em: {output_path}")

# Estatísticas finais
print(f"\n=== ESTATÍSTICAS FINAIS ===")
print(f"Total de registros: {len(df)}")
print(f"Total de colunas: {len(df.columns)}")
print(f"Categorias únicas: {df['CALENDAR'].nunique()}")
print(f"Títulos únicos: {df['TITLE'].nunique()}")

=== SALVANDO DATASET PROCESSADO ===
✅ Dataset processado salvo em: ../data/tasks_summary.csv

=== ESTATÍSTICAS FINAIS ===
Total de registros: 4035
Total de colunas: 7
Categorias únicas: 11
Títulos únicos: 215


In [72]:
# Mostrar amostra do dataset final
print("=== AMOSTRA DO DATASET FINAL ===")
print("Colunas do dataset final:")
print(df.columns.tolist())
print(f"\nPrimeiros 5 registros:")
print(df.head())
print(f"\nTipos de dados:")
print(df.dtypes)

=== AMOSTRA DO DATASET FINAL ===
Colunas do dataset final:
['SUMMARY', 'DTSTART', 'DTEND', 'CALENDAR', 'DURATION', 'CREATED', 'TITLE']

Primeiros 5 registros:
           SUMMARY             DTSTART               DTEND     CALENDAR  \
0    cafe da manha 2025-01-07 08:00:00 2025-01-07 08:20:00  Alimentação   
1  lanche da manha 2025-01-07 10:00:00 2025-01-07 10:05:00  Alimentação   
2           almoco 2025-01-07 12:30:00 2025-01-07 13:00:00  Alimentação   
3  lanche da tarde 2025-01-07 16:00:00 2025-01-07 16:15:00  Alimentação   
4       pre-treino 2025-01-07 18:00:00 2025-01-07 18:05:00  Alimentação   

   DURATION             CREATED                    TITLE  
0  0.333333 2025-01-07 10:12:00           Have breakfast  
1  0.083333 2025-01-07 10:14:00     Have a morning snack  
2  0.500000 2025-01-07 10:15:00               Have lunch  
3  0.250000 2025-01-07 10:16:00  Have an afternoon snack  
4  0.083333 2025-01-07 10:17:00         Take pre-workout  

Tipos de dados:
SUMMARY            